## Gsm8k majority voting

In [ ]:
import json
with open('/workspace/ye/Github/ds-chat/Shepherd/data/orm_data/test-set.json',"r",encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
voting_corr_num=0
random_corr=0
corr_num=0
false_num=0
from collections import Counter
for i in range(len(data)):
    binary_label = []
    match_result = []
    gt = data[i]['answer'].split("#### ")[-1]
    if data[i]['response'][0].strip("<endoftext>").split("#### ")[-1] == gt:
        random_corr+=1
    for j in data[i]['response']:
        match_result.append(j.strip("<endoftext>").split("#### ")[-1])
        if j.strip("<endoftext>").split("#### ")[-1]==gt:
            binary_label.append(1)
            corr_num+=1
        else:
            binary_label.append(0)
            false_num+=1
    counter = Counter(match_result)
    most_common_count = counter.most_common(1)[0][0]
    if most_common_count == gt:
        voting_corr_num+=1
    else:
        pass
    data[i]['labels'] = binary_label
    data[i]['match_result'] = match_result
    data[i]['voting_result'] = most_common_count
    data[i]['dataset']='Gsm8k'
print(f'Random select acc {random_corr/len(data)*100:.2f}%')
print(f'majority voting acc {voting_corr_num/len(data)*100:.2f}%')

## math majority voting

In [ ]:
import json
with open('/workspace/ye/Github/ds-chat/Shepherd/data/orm_data/test-set.json',"r",encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
voting_corr_num=0
random_corr=0
corr_num=0
false_num=0
from collections import Counter
import re
for i in range(len(data)):
    binary_label = []
    match_results = []
    gt = data[i]['answer']
    match_result = re.findall(r'\\boxed\{(.*)\}', data[i]['response'][0])
    if match_result !=[]:
        match_result=match_result[0]
    if match_result == gt:
        random_corr+=1
    # else:
    #     print(match_result)
    for j in data[i]['response']:
        match_result=re.findall(r'\\boxed\{(.*)\}', j)
        if match_result !=[]:
            match_result=match_result[0]
        else:
            match_result =None
        match_results.append(match_result)
        if match_result==gt:
            binary_label.append(1)
            corr_num+=1
        else:
            binary_label.append(0)
            false_num+=1
    # print(match_results)
    counter = Counter(match_results)
    most_common_count = counter.most_common(1)[0][0]
    if most_common_count == gt:
        voting_corr_num+=1
    # else:
    #     print(most_common_count,gt)
        pass
    data[i]['labels'] = binary_label
    data[i]['match_result'] = match_results
    data[i]['voting_result'] = most_common_count
    data[i]['dataset']='Math'
print(f'Random select acc {random_corr/len(data)*100:.2f}%')
print(f'majority voting acc {voting_corr_num/len(data)*100:.2f}%')

## ORM eval

In [ ]:
import json
with open('/workspace/ye/Github/ds-chat/Shepherd/infer_result/312-mistral-gsm8k-orm-80.65%.json',"r",encoding='utf-8') as f:
    data = json.load(f)

save_result = {}
for i in range(len(data)):
    if data[i]['prompt'] not in save_result:
        save_result[data[i]['prompt']] = {
            'response': [data[i]['response']],
            'pred_score': [data[i]['pred_score']],
            'answer': [data[i]['answer']]
        }
    else:
        save_result[data[i]['prompt']]['response'].append(data[i]['response'])
        save_result[data[i]['prompt']]['pred_score'].append(data[i]['pred_score'])
        save_result[data[i]['prompt']]['answer'].append(data[i]['answer'])

### gsm8k orm

In [ ]:
c = 0
all_c=0
t=0
for k,v in save_result.items():
    t+=1
    max_value_index = v['pred_score'].index(max(v['pred_score']))
    if v['answer'][0].split("#### ")[-1] == v['response'][max_value_index].strip("<endoftext>").strip("<step>").split("#### ")[-1].strip("</"):
        c+=1
    else:
        # print(v['answer'][0].split("#### ")[-1],'-------',v['response'][max_value_index].strip("<step>").split("#### ")[-1].strip("</"))
        pass
c/t

### gsm8k orm + self consistency

In [ ]:
c = 0
t=0
import re
for k,v in save_result.items():
    t+=1
    answers=[]
    for i in range(len(v['response'])):
        answers.append(v['response'][i].strip("<step>").strip("<endoftext>").strip("</s>").split("#### ")[-1])
    pred_scores =v['pred_score']
    cumulative_scores = {}
    for response, score in zip(answers, pred_scores):
        if response in cumulative_scores:
            cumulative_scores[response] += score
        else:
            cumulative_scores[response] = score
    highest_cumulative_value = max(cumulative_scores, key=cumulative_scores.get)
    
    if v['answer'][0].split("#### ")[-1] == highest_cumulative_value:
        c+=1
    else:
        # print(v['answer'][0].split("#### ")[-1],"---",highest_cumulative_value)
        pass
c/t

### math orm

In [ ]:
c = 0
t=0
import re
for k,v in save_result.items():
    t+=1
    max_value_index = v['pred_score'].index(max(v['pred_score']))
    math_result = re.findall(r'\\boxed\{(.*)\}', v['response'][max_value_index])
    if math_result==[]:
        math_result=[""]
    math_result = [v['response'][max_value_index].split("#### ")[-1].strip("</s>").strip("<step>")]
    if v['answer'][0]== math_result[0]:
        c+=1
    else:
        # print(v['answer'][0],'-------',math_result[0])
        pass
c/t

### math orm + self consistency

In [ ]:
import re
c = 0
t=0
for k,v in save_result.items():
    t+=1
    answers=[]
    for i in range(len(v['response'])):
        math_result = re.findall(r'\\boxed\{(.*)\}', v['response'][i])
        if math_result==[]:
            math_result=[""]
        answers.append(math_result[0])
    pred_scores =v['pred_score']
    cumulative_scores = {}
    for response, score in zip(answers, pred_scores):
        if response in cumulative_scores:
            cumulative_scores[response] += score
        else:
            cumulative_scores[response] = score
    highest_cumulative_value = max(cumulative_scores, key=cumulative_scores.get)
    # print(highest_cumulative_value)
    if v['answer'][0] == highest_cumulative_value:
        c+=1
    else:
        print(v['answer'][0],"====",highest_cumulative_value)
    
c/t